# Notebook 2.5 - Tạo tập dữ liệu nhiễu nền

## Cell 1: Tạo tập dữ liệu nhiễu nền
Cell này thực hiện việc tạo tập dữ liệu nhiễu nền cho lớp "unknown":
- Sử dụng class AudioPreprocessor từ notebook 2.0
- Đọc các file âm thanh nhiễu nền từ thư mục bg_noise
- Xử lý và tạo 150 file nhiễu nền mới:
  - Mỗi file dài 1 giây
  - Lấy ngẫu nhiên từ các file nhiễu nền gốc
  - Chuẩn hóa âm thanh
  - Lưu vào thư mục processed/unknown

Quy trình xử lý:
1. Đọc tất cả file nhiễu nền từ thư mục bg_noise
2. Với mỗi file:
   - Tải và chuyển đổi về 16kHz
   - Lấy ngẫu nhiên 1 đoạn 1 giây
   - Chuẩn hóa âm thanh
3. Nếu số lượng đoạn chưa đủ 150:
   - Lặp lại quá trình trên với các file ngẫu nhiên
4. Lưu tất cả các đoạn thành file WAV riêng biệt

Mục đích của notebook này là:
1. Tạo tập dữ liệu nhiễu nền cho lớp "unknown"
2. Đảm bảo tính đa dạng của dữ liệu nhiễu
3. Chuẩn hóa định dạng và chất lượng âm thanh
4. Chuẩn bị dữ liệu cho quá trình huấn luyện mô hình phân loại

In [1]:
import os
import numpy as np
import webrtcvad
import noisereduce as nr
from scipy.io import wavfile
from pydub import AudioSegment
import librosa
from tqdm import tqdm

class AudioPreprocessor:
    def __init__(self):
        self.vad = webrtcvad.Vad(2)
        self.target_sr = 16000
        self.frame_duration_ms = 30
        
    def load_audio(self, input_path):
        """Load and convert audio to mono 16kHz"""
        audio = AudioSegment.from_wav(input_path).set_channels(1).set_frame_rate(self.target_sr)
        raw_audio = np.array(audio.get_array_of_samples())
        rate = audio.frame_rate
        return raw_audio, rate
        
    def denoise(self, audio, sr):
        """Apply noise reduction"""
        denoised = nr.reduce_noise(y=audio.astype(np.float32), sr=sr)
        if sr != self.target_sr:
            denoised = librosa.resample(denoised, orig_sr=sr, target_sr=self.target_sr)
        return denoised, self.target_sr
        
    def apply_vad(self, audio, sr):
        """Apply Voice Activity Detection"""
        frame_length = int(sr * self.frame_duration_ms / 1000)
        frames = [audio[i:i+frame_length] for i in range(0, len(audio) - frame_length, frame_length)]

        def is_speech(frame):
            int16_frame = (frame * 32768).astype(np.int16)
            return self.vad.is_speech(int16_frame.tobytes(), sr)

        flags = [is_speech(frame) for frame in frames]
        speech_mask = np.repeat(flags, frame_length)
        speech_mask = np.pad(speech_mask, (0, len(audio) - len(speech_mask)), mode='constant')
        return audio * speech_mask
        
    def normalize_audio(self, audio):
        """Apply peak normalization"""
        max_val = np.max(np.abs(audio))
        if max_val > 0:
            audio = audio / max_val * 0.99
        return (audio * 32767).astype(np.int16)

# Configuration
# background_noise_dir = '../data/_background_noise_'
background_noise_dir = '../data/bg_noise'
output_dir = '../data/processed/unknown'
os.makedirs(output_dir, exist_ok=True)

# Number of files to generate
num_output_files = 150

# Initialize preprocessor
processor = AudioPreprocessor()

# List of background noise files
background_files = [f for f in os.listdir(background_noise_dir) if f.endswith('.wav')]

# Read and process all background audio
background_segments = []
for fname in tqdm(background_files, desc="Processing background files"):
    path = os.path.join(background_noise_dir, fname)
    samples, sr = librosa.load(path, sr=16000)
    
    # Only take a random 1s segment from each file
    if len(samples) >= sr:  # Make sure file is at least 1s long
        # Choose random start position
        max_start = len(samples) - sr
        start_idx = np.random.randint(0, max_start + 1)
        segment = samples[start_idx:start_idx + sr]
        
        # Apply preprocessing
        segment = processor.normalize_audio(segment)
        background_segments.append(segment)

# If number of segments is less than num_output_files, repeat until enough
while len(background_segments) < num_output_files:
    # Randomly select a file
    fname = np.random.choice(background_files)
    path = os.path.join(background_noise_dir, fname)
    samples, sr = librosa.load(path, sr=16000)
    
    if len(samples) >= sr:
        max_start = len(samples) - sr
        start_idx = np.random.randint(0, max_start + 1)
        segment = samples[start_idx:start_idx + sr]
        segment = processor.normalize_audio(segment)
        background_segments.append(segment)

# Write to files
for idx, segment in enumerate(tqdm(background_segments, desc="Saving unknown segments")):
    output_path = os.path.join(output_dir, f"noise_{idx}.wav")
    wavfile.write(output_path, processor.target_sr, segment)

print(f"Created {len(background_segments)} file noise in {output_dir}")

Saving unknown segments: 100%|██████████| 150/150 [00:00<00:00, 28108.19it/s]

Created 150 file noise in ../data/processed/unknown
